In [122]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [160]:
aicum = pd.read_excel("data/AICUM Vaccination survey 1.19.2021.xlsx")
columns = {
    "Institution":"institution",
    "Survey completed by:":"point_of_contact",
    "Enrolled in MCVP":"MCVP",
    "Submitted Application for MCVP":"submitted_mcvp_app",
    "Date of submission":"date_of_submission",
    "Partnering with hospital/healthcare provider":"partnering",
    "Named local hospital or healthcare provider":"partner_name",
    "Interested in serving as vaccination site/providing staffing and operational support":"vaccination_site_support",
    "Interested in serving as a vaccination site without providing staffing and operational support":"vaccination_site_no_support",
    "All eligible employees vaccinated":"employees_vaccinated",
    "on campus or external entity":"on_campus_or_external"
}
aicum = aicum.rename(columns=columns)

def add_college(x):
    exceptions = ["College", "University", "Institute", "Conservatory"]
    universities = ["Bay Path", "Bentley", "Brandeis", "Northeastern", "Suffolk", "Tufts"]
    
    misc_remaps = {
        'Anne Maria': 'Anna Maria College',  # typo in AICUM survey
        'Holy Cross': 'College of the Holy Cross',
        'MGH Institute': 'MGH Institute of Health Professions',
        'Montserrat': 'Montserrat College of Art',
        'Olin': 'Franklin W. Olin College of Engineering',
        'Urban': 'Urban College of Boston',
        'WPI': 'Worcester Polytechnic Institute',
        'Wentworth': 'Wentworth Institute of Technology',
        'NE Conservatory': "The New England Conservatory of Music"
    }
    
    if x in misc_remaps.keys():
        return misc_remaps[x]
    elif any(university in x for university in universities):
        return x + " University"
    elif any(exception in x for exception in exceptions):
        return x
    else:
        return x + " College"
aicum['institution'] = aicum['institution'].apply(add_college)
aicum_insts = set(aicum['institution'])
print(len(aicum_insts))

38


In [161]:
orig_colleges = gpd.read_file("data/MA_Universities/SHP_dormcap/ma_universities.shp")
# Remove some gratuitous listings, as in the spring 2020 analysis.
orig_colleges = orig_colleges[orig_colleges["COLLEGE"] != "University of Massachusetts Dartmouth Center for Innovation and Entrepreneurship"]
orig_colleges = orig_colleges[~((orig_colleges["COLLEGE"] == "Northeastern University") & (orig_colleges["CAMPUS"] != 'Main Campus'))]
orig_colleges = orig_colleges[~((orig_colleges["COLLEGE"] == "Boston College") & (orig_colleges["CAMPUS"] != 'Main Campus'))]

In [162]:
missing = gpd.read_file("data/MA_Universities/missing_AICUM/missing_AICUM.csv")

columns = {
    "Institution Name":"COLLEGE",
    "Street address or post office box (HD2019)":"ADDRESS",
    "City location of institution (HD2019)":"CITY",
    "ZIP code (HD2019)":"ZIPCODE",
    "Institution's internet website address (HD2019)":"URL",
    "12-month unduplicated headcount  total: 2018-19 (DRVEF122019)":"ENROLL",
    "Latitude location of institution (HD2019)":"LAT",
    "Longitude location of institution (HD2019)":"LON",
}

missing = missing.rename(columns=columns)
missing["geometry"] = [Point(float(x),float(y)) for x, y in zip(missing["LON"], missing["LAT"])]

In [163]:
colleges = orig_colleges.append(missing)

In [164]:
colleges = colleges.merge(aicum, left_on='COLLEGE', right_on='institution', how='inner')
matched_insts = set(colleges['COLLEGE'])
print(len(aicum_insts.difference(matched_insts)), "unmatched AICUM institutions")

0 unmatched AICUM institutions


In [166]:
filtered_colleges = colleges[
  ['COLLEGE', 'CAMPUS', 'ADDRESS', 'CITY', 'ZIPCODE', 'ENROLL',
   'URL', 'MCVP', 'submitted_mcvp_app', 'partner_name',
   'vaccination_site_support', 'vaccination_site_no_support',
   'employees_vaccinated', 'on_campus_or_external', 'geometry']
]

In [167]:
filtered_colleges = filtered_colleges.to_crs('epsg:4326')

In [174]:
filtered_colleges.set_index('COLLEGE').loc['Brandeis University']

CAMPUS                                                                 None
ADDRESS                                                    415 South Street
CITY                                                                Waltham
ZIPCODE                                                               02454
ENROLL                                                                 5800
URL                                                 http://www.brandeis.edu
MCVP                                                                     No
submitted_mcvp_app                                                       No
partner_name                                                            NaN
vaccination_site_support                                                 No
vaccination_site_no_support                                             Yes
employees_vaccinated                                                    Yes
on_campus_or_external                                       External Entity
geometry    

In [177]:
filtered_colleges.to_file('../covid-vaccines/geojson/ma_universities_with_aicum.geojson', driver='GeoJSON')

DriverError: Failed to create GeoJSON datasource: ../covid-vaccines/geojson/ma_universities_with_aicum.geojson: ../covid-vaccines/geojson/ma_universities_with_aicum.geojson: No such file or directory

# unsure: 
 * Bay Path — University, or Adult Evening? (Parker: University, apparently?)
 * Anne Maria — doesn't show up in dataset
 * Holy Cross — Hellenic College/Holy Cross, or College of the Holy Cross?  (Parker: almost certainly the latter)
 
 
# Missing from SHP_dormcap (which has enrollment...)
* Cambridge College
* Fisher College
* Laboure College (nursing?)
* MGH Institute of Health Professions
* New England Conservatory of Music
* Urban College of Boston
* William James College

These mainly appear to be specialty schools (in particular, most have graduate-level programs in psychology and nursing). We really only need locations (lat, long) and total enrollment—address information, etc. is just gravy—so we could try to pull these from the other shapefiles/GeoJSON files in `data` or—as a last resort—pull some information from Google Maps and school websites.